In [1]:
import tweepy
import time
import networkx as nx
import pandas as pd
import json
import numpy as np
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer
from datetime import datetime
from networkx.readwrite import json_graph
%matplotlib inline

In [2]:
consumer_key = "2e6dNamYAUwvPcLJ5YKn9ow6R"
consumer_secret = "VFujaQNDkF6d7YKrTttJzMhCfNpFcYCnMyDSvjGssEvq4WYvnm"
access_token = "939149568519815169-c6Lti1dhyk5Z8wmwymsrpvw1BtxVE0Z"
access_token_secret = "IaQwQlN3yrkoZsB5nzi8Ha2AO1TseVdLcPJgltP6ljZiI"   

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [17]:
def constructGraph(screen_name, max_followers):
    G = nx.DiGraph()
    res = api.get_user(screen_name)
    G.add_node(res.id, attr_dict = {'status_count': res.statuses_count, 'screen_name': res.screen_name})
    for user in tweepy.Cursor(api.followers, id=res.id).items(max_followers):
        G.add_node(user.id, attr_dict = {'status_count': user.statuses_count, 'screen_name': user.screen_name})
        G.add_edge(user.id, res.id, retweets=0)
    return G
    

In [18]:
G = constructGraph('nasw', 150)

Rate limit reached. Sleeping for: 418


In [19]:
def saveGraph(G, filename):
    data = json_graph.node_link_data(G)
    json_data = json.dumps(data)
    with open(filename, 'w') as file:
        file.write(json_data)
        
def loadGraph(filename):        
    with open(filename, 'r') as file:
        json_data = file.read()
        data2 = json.loads(json_data)
    G = json_graph.node_link_graph(data2)
    return G

In [20]:
saveGraph(G, 'retweetGraph.json')

In [69]:
def fetchRatio(G, screen_name, max_tweets, max_retweets):
    for edge in G.edges():
        G[edge[0]][edge[1]]['retweets'] = 0
    res = api.get_user(screen_name)    
    for tweet in tweepy.Cursor(api.user_timeline, id=res.id).items(max_tweets):
        retweets = api.retweets(id=tweet.id, count=max_retweets)
        for retweet in retweets:
            retweet_id = retweet.user.id
            if retweet_id in G:
                G.add_edge(retweet_id, res.id, retweets=G[retweet_id][res.id]['retweets'] + 1)
    for node in G.nodes():
        if res.id != node:
            G.add_edge(node, res.id,ratio = (G[node][res.id]['retweets'] / max_tweets))
    return G

In [22]:
G = fetchRatio(G, 'nasw', 100, 100)

Rate limit reached. Sleeping for: 797


In [27]:
res = api.get_user("Shalisa_Renee")

In [29]:
G.add_node(res.id, attr_dict = {'status_count': res.statuses_count, 'screen_name': res.screen_name})

In [36]:
G.add_edge(res.id, 17869267, retweets=0)

In [70]:
G = fetchRatio(G, 'nasw', 3, 50)

In [72]:
saveGraph(G, 'retweetGraph.json')

In [73]:
G = loadGraph('retweetGraph.json')

In [77]:
sum = 0
for edge in G.edges():
    sum += G[edge[0]][edge[1]]['ratio']
print(sum / len(G.edges()))

0.006622516556291391
